# **Automatically tag support tickets into categories using a large language model (LLM).**

# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --upgrade --no-cache-dir --verbose

In [2]:
!pip install huggingface-hub

# Cell A
  hf_hub_download is used to download pre-trained models and files from the Hugging Face Hub*

  Llama from llama_cpp is used to load and run a LLaMA-based large language model locally for inference tasks such as text classification.

In [3]:
# Import the 'hf_hub_download' function from the 'huggingface_hub' library
from huggingface_hub import hf_hub_download

# Import the 'Llama' class from the 'llama_cpp' library
from llama_cpp import Llama


# Cell B
 model_name_or_path specifies the Hugging Face repository that hosts the LLaMA-2-13B chat model in GGUF format.

 model_basename selects the specific quantized GGUF model file to load (Q6_K), balancing model accuracy and memory efficiency for local inference.

In [4]:
# Define the model name or path as a string (You can find this info from hugging face website)

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"

# Define the model basename as a string, indicating it's in the gguf format

#model_basename =  "llama-2-13b-chat.Q5_K_M.gguf"

model_basename = "llama-2-13b-chat.Q6_K.gguf"

# Cell C
This code downloads the specified quantized LLaMA model file from the Hugging Face Hub.

repo_id identifies the model repository, while filename selects the exact GGUF model file.

The downloaded file path is stored in model_path for later loading and inference.

In [5]:

# Download the model from the Hugging Face Hub using the 'hf_hub_download' function
# by specifying the 'repo_id' and 'filename'

model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Cell D
Initializes the LLaMA model for local inference

Uses CPU threads and GPU layers for optimized performance

Sets a large context window for handling long support tickets

In [6]:
# Create an instance of the 'Llama' class with specified parameters
# remove the blank spaces and complete the code

lcpp_llm = Llama(
        model_path=model_path,
        n_threads=2,  # CPU cores
        n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
        n_ctx=4096,  # Context window
    )



AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
# Cell E

Write a Python function called **generate_llama_response** that takes a single parameter, support_ticket_text, which represents the user's support ticket text. Inside the function, you should perform the following tasks:

Define a system message as a string and assign it to the variable system_message.

- **Combine the support_ticket_text and system_message to create a prompt string.**

*Generate a response from the LLaMA model using the lcpp_llm instance with the following parameters:*

- prompt should be the combined prompt string.
- max_tokens should be set to 256.
- temperature should be set to 0.
- top_p should be set to 0.95.
- repeat_penalty should be set to 1.2.
- top_k should be set to 50.
- stop should be set as a list containing 'INST'.
- echo should be set to False.
Extract and return the response text from the generated response.

Don't forget to provide a value for the system_message variable before using it in the function.



# Cell F
#  Description Below

- ** Introduction (System Role):** Begin with an introductory statement that establishes the role of the system message. In this case, it's acting as a guide for a technical assistant.

- ** Ticket Categorization:** Explain the primary task of the technical assistant, which is to classify the support ticket into specific categories. In this example, the categories are:
    - Technical Issues
    - Hardware Issues
    - Data Recovery

- ** Response Options:** Clearly state that the assistant should only respond with one of the predefined categories, emphasizing that other responses are not acceptable.

- ** Sub-Tasks:** Outline the secondary tasks that the technical assistant should perform once the category is identified. These sub-tasks include:
  - ** Creating Tags:** Instruct the assistant to create tags that will help further classify the ticket.
  - ** Assigning Priority:** Specify that the assistant should assign a priority level (e.g., "High" or "Normal") based on their understanding of the text.
  - ** Suggesting ETA:** Guide the assistant to provide an estimated time for
resolving the issue mentioned in the ticket.
  - ** Generating 1st Reply (Sentiment-Based):** Emphasize the importance of crafting a response that aligns with the sentiment expressed in the ticket.

- ** General Instructions:** Offer general instructions that should be followed throughout the ticket processing, such as:

  - ** Categorization:** Reiterate that the assistant should categorize the ticket only into the predefined categories.
  - ** Reading Carefully:** Stress the importance of reading the support ticket text thoroughly and considering the overall sentiment before responding.
  - ** Tone:** Emphasize that the tone of all responses should be polite and professional.
  - ** Output Format:** Clearly specify the desired output format for the responses generated by the assistant. In this case, the output should be in JSON format.

##### **The output of the model should be in JSON format**

In [198]:
def generate_llama_response(support_ticket_text):

    # System message
    system_message = """
    [INST]<<SYS>> You are acting as a guide for a technical assistant. As a technical assistant, your primary task is to classify the support ticket into specific
    categories. There are three categories:Technical Issues, Hardware Issues, and Data Recovery.

    As a technical assistant, you should only respond with one of the predefined categories (Technical Issues, Hardware Issues, and Data Recovery). Other responses are not acceptable.

    Your goal is to identify the category and then provide the following information from the support ticket text, including the creating tags, assigning priority, suggested ETA,
    generating 1st reply based on sentiment. Go through the each support ticket text thoroughly and considering the overall sentiment before responding, and generate only a structured output for
    the JSON format.
    Here is the structured output for further analysis in JSON format. Follow the JSON format below strictly.You must include the following as JSON output:


    {"category": <create one of the predefined categories (Technical Issues, Hardware Issues, and Data Recovery). Other responses are not acceptable>,
     "tags": <create tags to further classify the ticket>,
     "priority": <assign a priority level (e.g., "High" or "Normal") only based on the understanding of the text>,
     "suggested_ETA": <provide an estimated time for resolving the issue mentioned in the ticket>,
     "generated_1st_reply": <generate a 50 words reply that aligns with the sentiment expressed in the ticket.The tone of all responses should be polite and professional.>
     }
     
     Here is an example:

     support_ticket_text: My internet connection is very slow for the last 3 days. Therefore, it is hard for me to work efficiently from home.
     I am also facing frequent disconnections. Can you please help me to resolve this issue promptly.

     Your output should be:

     {"category": "Technical Issues",
      "tags": ["connectivity", "slow internet"],
      "priority": "High",
      "suggested_ETA": "ASAP",
      "generated_1st_reply": "We are sorry to hear the slow internet connectivity issues. Our team will work promptly to resolve this issue and get your connection back up to speed."



    <</SYS>>[/INST]
    """

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{support_ticket_text}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=256,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]  ### Fill in the blank
    return response_text



# CelL G
   Mount Google Drive

### Cell Description: 
   This cell mounts the user's Google Drive to the Colab environment, allowing access to files stored in Drive at '/content/drive'.

In [8]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Cell H
   Load Support Ticket Data

### Cell Description:
This cell imports the pandas library and reads the CSV file containing support ticket text data from Google Drive into a pandas DataFrame named 'data' for further analysis.

In [199]:
# Import the pandas library and alias it as 'pd'

# Read a CSV file into a DataFrame and store it in the 'data' variable
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Support_ticket_text_data_mid_term.csv')


In [200]:
data.head()

,support_tick_id,support_ticket_text
0,ST2023-006,My internet connection has significantly slowe...
1,ST2023-007,Urgent help required! My laptop refuses to sta...
2,ST2023-008,I've accidentally deleted essential work docum...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...
4,ST2023-010,"My smartphone battery is draining rapidly, eve..."


# Cell I
## Cell name: 
Generate LLaMA Responses

### Cell Description:
This cell applies the 'generate_llama_response' function to each entry in the 'support_ticket_text' column of the DataFrame 'data'. The generated responses are stored in a new column called 'llama_response' for further analysis or processing.

In [201]:
# Apply a function to each element in the 'support_ticket_text' column of the DataFrame 'data'
# The applied function, in this case, is a lambda function.

# The lambda function takes a single argument 'x', representing each individual 'support_ticket_text' in the column.

# Inside the lambda function:
# - 'generate_llama_response(x)' is called to generate a response based on the 'support_ticket_text'.
# - The result of 'generate_llama_response(x)' is assigned to a new column called 'llama_response' in the DataFrame 'data'.

# Example - data['llama_response'] = data['support_ticket_text'].apply(lambda x: generate_llama_response(x))
data['llama_response'] = data['support_ticket_text'].apply(lambda x: generate_llama_response(x))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


# Cell J
    Check LLaMA Response Data Type

### Cell Description:
     This cell checks and displays the data type of the 'llama_response' column in the DataFrame 'data' to ensure the responses were stored in the expected format (e.g., string/object).

In [202]:
data['llama_response'].dtypes

dtype('O')

# Cell K
    Preview First LLaMA Response

### Cell Description:
     This cell displays the first generated response from the 'llama_response' column in the DataFrame 'data' to verify the output of the 'generate_llama_response' function.

In [203]:
data['llama_response'][0]

' Sure, I\'d be happy to help! Based on the support ticket text you provided, I would categorize this issue as a Technical Issue. Here is the structured output for further analysis in JSON format:\n\n{\n"category": "Technical Issues",\n"tags": ["internet connection slow"],\n"priority": "High",\n"suggested_ETA": "ASAP",\n"generated_1st_reply": "Sorry to hear that you\'re experiencing issues with your internet connection. Our team is here to help and will work promptly to resolve the issue."\n}'

# Cell L
    Preview DataFrame with LLaMA Responses
### Cell Description: 
    This cell displays the first few rows of the DataFrame 'data', allowing you to verify that the new column 'llama_response' has been successfully added alongside the original 'support_ticket_text' column.

In [204]:
## Check the new_column added
data.head()

,support_tick_id,support_ticket_text,llama_response
0,ST2023-006,My internet connection has significantly slowe...,"Sure, I'd be happy to help! Based on the supp..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"Sure, I'd be happy to help! Based on the supp..."
2,ST2023-008,I've accidentally deleted essential work docum...,"Sure, I'd be happy to help! Based on the supp..."
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"Sure, I'd be happy to help! Based on the supp..."
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","Sure, I'd be happy to help! Based on the supp..."


# Cell M 
   Check Number of LLaMA Responses

## Cell Description: 
    This cell prints the total number of entries in the 'llama_response' column of the DataFrame 'data', helping to confirm that a response has been generated for each support ticket.

In [205]:
print(len(data['llama_response']))

21


# Cell N
   Extract JSON from First LLaMA Response

In [206]:
#Trimming texts from the first llama_response, keeping only the JSON response
import re
updated_response= re.findall(r'\{([^}]*)\}', data['llama_response'][0])
print(updated_response)

['\n"category": "Technical Issues",\n"tags": ["internet connection slow"],\n"priority": "High",\n"suggested_ETA": "ASAP",\n"generated_1st_reply": "Sorry to hear that you\'re experiencing issues with your internet connection. Our team is here to help and will work promptly to resolve the issue."\n']


# Cell O
  Trim LLaMA Responses to JSON
## Cell Description: 
    This cell iterates over the first 21 entries of the 'llama_response' column in the DataFrame 'data', extracts only the JSON content using a regular expression, reformats it as a proper JSON string, updates the column, and prints each trimmed response for verification.

In [207]:
#Trimming texts from the llama_response, keeping only the JSON response
import re
for i in range(0,21):
  data['llama_response'][i] = re.findall(r'\{([^}]*)\}', data['llama_response'][i])[0]
  data['llama_response'][i] = f"{{{data['llama_response'][i]}}}"
  print(data['llama_response'][i])

{
"category": "Technical Issues",
"tags": ["internet connection slow"],
"priority": "High",
"suggested_ETA": "ASAP",
"generated_1st_reply": "Sorry to hear that you're experiencing issues with your internet connection. Our team is here to help and will work promptly to resolve the issue."
}
{
"category": "Hardware Issues",
"tags": ["Laptop", "Startup", "Restart"],
"priority": "High",
"suggested_ETA": "24 hours",
"generated_1st_reply": "Sorry to hear that your laptop is not starting up. We understand the urgency of this issue and will prioritize it accordingly."
}
{
"category": "Data Recovery",
"tags": ["deleted documents", "data loss"],
"priority": "High",
"suggested_ETA": "ASAP",
"generated_1st_reply": "Sorry to hear that you've lost essential work documents. Our team will prioritize the data recovery process and guide you through it as soon as possible."
}
{
"category": "Technical Issues",
"tags": ["Wi-Fi signal strength", "router placement"],
"priority": "Normal",
"suggested_ETA": "2

In [208]:
print(data['llama_response'].dtype)

object


In [209]:
data['llama_response'][0]

'{\n"category": "Technical Issues",\n"tags": ["internet connection slow"],\n"priority": "High",\n"suggested_ETA": "ASAP",\n"generated_1st_reply": "Sorry to hear that you\'re experiencing issues with your internet connection. Our team is here to help and will work promptly to resolve the issue."\n}'

In [210]:
data.head()

,support_tick_id,support_ticket_text,llama_response
0,ST2023-006,My internet connection has significantly slowe...,"{\n""category"": ""Technical Issues"",\n""tags"": [""..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{\n""category"": ""Hardware Issues"",\n""tags"": [""L..."
2,ST2023-008,I've accidentally deleted essential work docum...,"{\n""category"": ""Data Recovery"",\n""tags"": [""del..."
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{\n""category"": ""Technical Issues"",\n""tags"": [""..."
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{\n""category"": ""Technical Issues"",\n""tags"": [""..."


In [211]:
import json

# Cell P
   Parse LLaMA Responses as JSON

## Cell Description:
   This cell defines a function 'extract_json_data' that parses a JSON-formatted string and converts it into a Python dictionary. The function is then applied to the 'llama_response' column of the DataFrame 'data', creating a new column 'llama_response_parsed' that stores the parsed JSON data for easier access to individual key-value pairs.

In [212]:
# Function to parse JSON data and extract key-value pairs
def extract_json_data(json_str):
    try:
        data_dict = json.loads(json_str)
        return data_dict
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

# Apply the function to the 'llama_response' column
data['llama_response_parsed'] = data['llama_response'].apply(extract_json_data)

In [213]:
data

,support_tick_id,support_ticket_text,llama_response,llama_response_parsed
0,ST2023-006,My internet connection has significantly slowe...,"{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['int..."
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{\n""category"": ""Hardware Issues"",\n""tags"": [""L...","{'category': 'Hardware Issues', 'tags': ['Lapt..."
2,ST2023-008,I've accidentally deleted essential work docum...,"{\n""category"": ""Data Recovery"",\n""tags"": [""del...","{'category': 'Data Recovery', 'tags': ['delete..."
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['Wi-..."
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['sma..."
5,ST2023-011,I'm locked out of my online banking account an...,"{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['pas..."
6,ST2023-012,"My computer's performance is sluggish, severel...","{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['per..."
7,ST2023-013,I'm experiencing a recurring blue screen error...,"{""category"": ""Hardware Issues"",\n""tags"": [""blu...","{'category': 'Hardware Issues', 'tags': ['blue..."
8,ST2023-014,My external hard drive isn't being recognized ...,"{\n""category"": ""Data Recovery"",\n""tags"": [""Ext...","{'category': 'Data Recovery', 'tags': ['Extern..."
9,ST2023-015,The graphics card in my gaming laptop seems to...,"{\n""category"": ""Hardware Issues"",\n""tags"": [""g...","{'category': 'Hardware Issues', 'tags': ['grap..."


In [214]:
data['llama_response_parsed'][0]

{'category': 'Technical Issues',
 'tags': ['internet connection slow'],
 'priority': 'High',
 'suggested_ETA': 'ASAP',
 'generated_1st_reply': "Sorry to hear that you're experiencing issues with your internet connection. Our team is here to help and will work promptly to resolve the issue."}

In [215]:
# Concatenate the original DataFrame 'data' with a new DataFrame created by normalizing JSON data.

# The 'data['llama_response_parsed']' column is assumed to contain JSON data that needs to be flattened and normalized.

# 'pd.json_normalize' is a pandas function used to normalize semi-structured JSON data into a flat DataFrame.
# In this case, it's applied to the 'llama_response_parsed' column, which presumably contains JSON data.

# The result of 'pd.json_normalize' is a DataFrame where the JSON data is flattened and each element becomes a separate column.

# The 'axis=1' parameter specifies that the concatenation should be done horizontally, i.e., the new columns from normalization
# will be added as new columns in the 'data' DataFrame.

# After this operation, the 'data' DataFrame will contain the original columns along with the additional columns
# generated by normalizing the JSON data from the 'llama_response_parsed


# example - data = pd.concat([data, pd.json_normalize(data['llama_response_parsed'])], axis=1)
data = pd.concat([data, pd.json_normalize(data['llama_response_parsed'])], axis=1)

In [216]:
data

,support_tick_id,support_ticket_text,llama_response,llama_response_parsed,category,tags,priority,suggested_ETA,generated_1st_reply
0,ST2023-006,My internet connection has significantly slowe...,"{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['int...",Technical Issues,[internet connection slow],High,ASAP,Sorry to hear that you're experiencing issues ...
1,ST2023-007,Urgent help required! My laptop refuses to sta...,"{\n""category"": ""Hardware Issues"",\n""tags"": [""L...","{'category': 'Hardware Issues', 'tags': ['Lapt...",Hardware Issues,"[Laptop, Startup, Restart]",High,24 hours,Sorry to hear that your laptop is not starting...
2,ST2023-008,I've accidentally deleted essential work docum...,"{\n""category"": ""Data Recovery"",\n""tags"": [""del...","{'category': 'Data Recovery', 'tags': ['delete...",Data Recovery,"[deleted documents, data loss]",High,ASAP,Sorry to hear that you've lost essential work ...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,"{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['Wi-...",Technical Issues,"[Wi-Fi signal strength, router placement]",Normal,24 hours,Thank you for reaching out to us with your Wi-...
4,ST2023-010,"My smartphone battery is draining rapidly, eve...","{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['sma...",Technical Issues,[smartphone battery drain],High,24 hours,Sorry to hear that your smartphone's battery i...
5,ST2023-011,I'm locked out of my online banking account an...,"{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['pas...",Technical Issues,[password reset],High,ASAP,Sorry to hear that you're having trouble acces...
6,ST2023-012,"My computer's performance is sluggish, severel...","{\n""category"": ""Technical Issues"",\n""tags"": [""...","{'category': 'Technical Issues', 'tags': ['per...",Technical Issues,"[performance, sluggish]",High,ASAP,Sorry to hear that your computer's performance...
7,ST2023-013,I'm experiencing a recurring blue screen error...,"{""category"": ""Hardware Issues"",\n""tags"": [""blu...","{'category': 'Hardware Issues', 'tags': ['blue...",Hardware Issues,"[blue screen error, crashes]",High,2-3 business days,Sorry to hear that you're experiencing frequen...
8,ST2023-014,My external hard drive isn't being recognized ...,"{\n""category"": ""Data Recovery"",\n""tags"": [""Ext...","{'category': 'Data Recovery', 'tags': ['Extern...",Data Recovery,"[External Hard Drive, Recovery, Vital Data]",High,2-3 business days,Sorry to hear that you're experiencing issues ...
9,ST2023-015,The graphics card in my gaming laptop seems to...,"{\n""category"": ""Hardware Issues"",\n""tags"": [""g...","{'category': 'Hardware Issues', 'tags': ['grap...",Hardware Issues,"[graphics card, poor gaming performance]",High,2-3 business days,Sorry to hear that you're experiencing poor ga...


In [217]:
# Drop specific columns which are not needed from the DataFrame 'data'
# Keep the columns which are mentioned in the sample output
del data['llama_response']

In [218]:
del data['llama_response_parsed']

In [219]:
data.head()

,support_tick_id,support_ticket_text,category,tags,priority,suggested_ETA,generated_1st_reply
0,ST2023-006,My internet connection has significantly slowe...,Technical Issues,[internet connection slow],High,ASAP,Sorry to hear that you're experiencing issues ...
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware Issues,"[Laptop, Startup, Restart]",High,24 hours,Sorry to hear that your laptop is not starting...
2,ST2023-008,I've accidentally deleted essential work docum...,Data Recovery,"[deleted documents, data loss]",High,ASAP,Sorry to hear that you've lost essential work ...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,Technical Issues,"[Wi-Fi signal strength, router placement]",Normal,24 hours,Thank you for reaching out to us with your Wi-...
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",Technical Issues,[smartphone battery drain],High,24 hours,Sorry to hear that your smartphone's battery i...




---



The parsed 'llama_response_parsed' column contains structured JSON data extracted from the LLaMA model responses. Each row now holds a dictionary of key-value pairs representing the model's analysis of the support ticket text. This allows for easier data exploration, such as identifying common issues, response categories, or extracting specific fields for reporting and analytics.